In [1]:
import pandas as pd
from numpy import nan as NA
import numpy as np

Get the filename to be loaded

In [2]:
file_name = str(input("enter the name of the file: "))

enter the name of the file: MHC-537-17 Classified Junction Count - All Sites


Load file from input.

In [3]:
data_combined = pd.concat(pd.read_excel(file_name + '.xlsx',ignore_index=True, sheet_name=None, header=26))
data_raw = pd.read_excel(file_name + '.xlsx', sheet_name=None, ignore_index=True)

Create a list with the sheetnames.

In [4]:
sheet_list = list(data_raw.keys())

Create a list containing only the movements (and thus number of movements) from the sheetnames

In [5]:
movement_list = []
for sheet in sheet_list:
        if 'Movement' in sheet:
            movement_list.append(sheet)
number_of_movements = len(movement_list)

Create a series containing only the PCU totals, per 15 minute interval.

In [6]:
remove_variables = ['Hourly Total', 'Hourly Average', 'Session Total', 'Session Average', 'TIME']

for i in remove_variables:
    data_combined = data_combined.drop(data_combined[(data_combined['TIME'] == i)].index)
    
cleaned_PCU_totals = data_combined['PCU TOTAL'].dropna()

Reshape the series as an array of columns as 15 minute time intervals and rows as the movements.

In [7]:
arr_totals = np.array(cleaned_PCU_totals)
number_of_time_intervals = len(arr_totals)/number_of_movements
organised_PCU_totals = arr_totals.reshape(number_of_movements, int(number_of_time_intervals))

Identify the indices at which to split the array, such that AM, inter and PM peaks can be found.

In [8]:
number_of_time_intervals = int(input("Enter the number of intervals: "))
split_locations = []
split_index = 0
for interval in range(number_of_time_intervals-1):
    split_index = (int(input("How many hours are there in interval " + str(interval+1) + ": "))*4) + split_index
    split_locations.append(split_index)

split_locations
survey_intervals = np.hsplit(organised_PCU_totals,split_locations)

Enter the number of intervals: 3
How many hours are there in interval 1: 2
How many hours are there in interval 2: 2


Total the columns (15 minute intervals), then calculate the rolling hour sum (Four consequetive columns). Drop null values which are the first three times.

In [9]:
rolling_hour_totals =[]
for i in survey_intervals:
    quarterly_sum = np.sum(i , axis=0)
    quarterly_sum_df = pd.DataFrame(quarterly_sum)
    rolling_hour_totals.append(quarterly_sum_df.rolling(4,).sum().dropna())

Display rolling hour PCU total, for the 15 minute intervals in the data, the first non-zero value is the first time interval in the data.

In [10]:
print( "The rolling hour totals, per 15 minute time interval (in PCUs) are: \n" + str(rolling_hour_totals))

The rolling hour totals, per 15 minute time interval (in PCUs) are: 
[         0
3  14438.0
4  14796.4
5  14949.5
6  15223.0
7  15647.3,          0
3  15443.5
4  15374.3
5  15166.7
6  14845.1
7  14950.1,          0
3  15574.9
4  15864.3
5  16068.9
6  16581.5
7  16560.2]


Get the maximum values, i.e. peaks. 

In [11]:
print("The peak values are: ")
for survey in rolling_hour_totals:
    x=np.array(survey)
    print(x.max())

The peak values are: 
15647.3
15443.5
16581.5
